IMDB란 영화리뷰에서, 긍정과 부정이 있는데 이 값을 한쌍의 데이터 값으로 만든 것이다. 50000개 중 25000개를 트레이닝, 나머지 25000개는 테스팅 샘플이다. 긍정 or 부정이라는 이진 특성을 갖고 있으므로, 바이너리 분류를 목적으로 한다.

여기서는 가장 자주 사용되는 20000개의 단어만 포함했다.

In [1]:
from keras.datasets import imdb

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=20000)


Using TensorFlow backend.


25000개의 트레이닝 샘플을, 20000 / 5000으로 쪼개 각각 트레이닝샘플/트레이닝 이후 테스트 샘플로 나누었다.

In [2]:
x_val = x_train[20000:]
y_val = y_train[20000:]
x_train = x_train[:20000]
y_train = y_train[:20000]

밑에서 사용되는 sequence.pad_sequences는, 하나의 리뷰 샘플의 크기를 200이라는 일정한 크기로 맞춰주기 위해 사용하였다.

In [3]:
from keras.preprocessing import sequence

x_train = sequence.pad_sequences(x_train, maxlen=200)
x_val = sequence.pad_sequences(x_val, maxlen=200)
x_test = sequence.pad_sequences(x_test, maxlen=200)

임베딩을 통해 단어간의 연관성을 파악한 벡터가 만들어지기 때문에 훈련하는데 도움이 된다.

sigmoid함수는 지금 이진 분류를 하기 때문에 사용했다.

dropout은 과대적합을 방지하기 위해 추가한 층이다.

Conv1D은 2차원에서도 쓰이지만, 1차원에서도 쓰일 수 있다. RNN보다는 계산 비용이 더 적게 들고 다양한 패턴을 인식할 수 있다. 하지만 연산량이 많아지는 문제점이 있다.

Pooling은 벡터의 특징적인 값 하나를 추출하는 과정이다. 이를 통해 과대적합을 해결하는데 많은 기여를 할 수 있다. GlobalMaxPooling은 가장 큰 특징적인 값을 가져오는 pooling이다.

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.layers import Dropout
from keras.layers import Conv1D, GlobalMaxPooling1D


model = Sequential()
model.add(Embedding(20000, 128, input_length=200))
model.add(Dropout(0.2))
model.add(Conv1D(256,
                 3,
                 padding='valid',
                 activation='relu',
                 strides=1))

model.add(GlobalMaxPooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [5]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 200, 128)          2560000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 200, 128)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 198, 256)          98560     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total para

In [6]:
hist = model.fit(x_train, y_train, epochs=25, batch_size=64, validation_data=(x_val, y_val))

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 20000 samples, validate on 5000 samples
Epoch 1/25
20000/20000 [==============================] - 105s 5ms/step - loss: 0.4660 - acc: 0.7590 - val_loss: 0.3039 - val_acc: 0.8708
Epoch 2/25
20000/20000 [==============================] - 103s 5ms/step - loss: 0.2161 - acc: 0.9143 - val_loss: 0.2644 - val_acc: 0.8904
Epoch 3/25
20000/20000 [==============================] - 103s 5ms/step - loss: 0.0847 - acc: 0.9731 - val_loss: 0.3088 - val_acc: 0.8876
Epoch 4/25
20000/20000 [==============================] - 103s 5ms/step - loss: 0.0228 - acc: 0.9947 - val_loss: 0.3573 - val_acc: 0.8956
Epoch 5/25
20000/20000 [==============================] - 104s 5ms/step - loss: 0.0066 - acc: 0.9992 - val_loss: 0.4069 - val_acc: 0.8946
Epoch 6/25
20000/20000 [==============================] - 103s 5ms/step - loss: 0.0027 - acc: 0.9997 - val_loss: 0.4452 - val_acc: 0.88

In [8]:
result=model.evaluate(x_test, y_test)

25000/25000 [==============================] - 40s 2ms/step


In [9]:
print(result[0])

0.5949737614390254


In [10]:
print(result[1])

0.86788
